<a href="https://colab.research.google.com/github/Ayman850/COMP257/blob/main/COMP257_Ass4_AymanYasin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment4 - Question 1

## Imports

In [ ]:
#1st I imported all necessaries import for this assignemnt i need, basically for this assignment we are using GaussianMixture.

import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.ndimage import rotate

## 1. Retrieve and load the Olivetti faces dataset

In [ ]:
# 1st step is to Retrieve and load the Olivetti faces dataset which is the same as previous assignment (ass2 and ass3)

olivetti_faces = fetch_olivetti_faces()
X = olivetti_faces.data
y = olivetti_faces.target
images = olivetti_faces.images # 64x64 version of .data

# here we printed the shape to see if its correct

print(X.shape)
print(y.shape)

## 2. Split the dataset into training, validation, and test sets using stratified sampling to ensure that each set contains the same number of images per person.

In [ ]:
# Here I split the dataset like previous ass ( Split is 80% train, 10% validation, and 10% test)

# train has 8 images for 1 person, 1 images for validation, and 1 images for test)

'''
Split is:
80% train
10% validation
10% test
to ensure we have a lot of data to train,
and also still have some left for validation and testing
'''
# First split into train + validation (90%) and test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

# Now split the train + validation set (90%) into train (80%) and validation (10%)
# 10/90 = 0.1111
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val)

In [ ]:
# here we print the shape again to see if its correct

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

## 3. Apply PCA on the training data, preserving 99% of the variance, to reduce the dataset’s dimensionality.

In [ ]:
# here I used PCA to preserve 99% of the variance, and that is by putting n_components=0.99.

pca = PCA(n_components=0.99)

X_train_reduced = pca.fit_transform(X_train)

In [ ]:
# here we found that it reduced 222 feature from 4096

print(X_train_reduced.shape)
print(y_train.shape)

## 4. Determine the most suitable covariance type for the dataset.

In [ ]:
# now we are trying to find most suitable covariance type for the dataset, and i did this by using AIC and BIC scores, for this i set the n_components = 20, and then i look through all the covariance_types.

covariance_types = ["spherical", "diag", "tied", "full"]

aic_scores_4 = []
bic_scores_4 = []
n_components = 20
n_inits = 10

for c_type in covariance_types:
    gm = GaussianMixture(
        n_components=n_components,
        covariance_type=c_type,
        n_init=n_inits,
        random_state=42)
    gm.fit(X_train_reduced)

    # Get scores
    aic = gm.aic(X_train_reduced)
    bic = gm.bic(X_train_reduced)

    # Collect scores
    aic_scores_4.append(aic)
    bic_scores_4.append(bic)

    print(f"Covariance type: {c_type}; AIC: {aic}; BIC: {bic}")


# At the end we find the best covariance type, and i print it, and from results it seems that both this scores "diag" is our best covariance type.

best_cov_aic = covariance_types[aic_scores_4.index(min(aic_scores_4))]
best_cov_bic = covariance_types[bic_scores_4.index(min(bic_scores_4))]

print(f"Best covariance type by AIC: {best_cov_aic}")
print(f"Best covariance type by BIC: {best_cov_bic}")

## 5. Determine the minimum number of clusters that best represent the dataset using either AIC or BIC.

In [ ]:
# now as we found our best covariance type which is diag, we used this again to find the minimum number of clusters where I use the range from 1-21, so it's basically I find 1 upto 20 to see which one is the best

covariance_type = "diag"
n_components_range = range(1, 21)
n_inits = 10
aic_scores_5 = []
bic_scores_5 = []

# this is the same method again basically

for i in n_components_range:
    gm = GaussianMixture(
        n_components=i,
        covariance_type=covariance_type,
        n_init=n_inits,
        random_state=42)
    gm.fit(X_train_reduced)

    # Get scores
    aic = gm.aic(X_train_reduced)
    bic = gm.bic(X_train_reduced)

    # Collect scores
    aic_scores_5.append(aic)
    bic_scores_5.append(bic)

    print(f"# of cluster: {i}; AIC: {aic}; BIC: {bic}")

# Then we find the best (scores) number of cluster

best_n_aic = n_components_range[aic_scores_5.index(min(aic_scores_5))]
best_n_bic = n_components_range[bic_scores_5.index(min(bic_scores_5))]

# Then we print the best number of cluster

print(f"Best # of clusters by AIC: {best_n_aic}")
print(f"Best # of clusters by BIC: {best_n_bic}")

# here I found the number of clusters is AIC: 3 and, BIC: 2

## 6. Plot the results from steps 3, 4, and 5.

In [ ]:
# Now I Ploting the results from steps 3, 4, and 5. and here steps 3 is for PCA, here we can see the highest point is 222 in order to reach 0.99

# Step 3 results
cumulative_variance = pca.explained_variance_ratio_.cumsum()
max_x = X_train_reduced.shape[1]
max_y = cumulative_variance[max_x - 1]

plt.figure(figsize=(8, 4))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance by PCA Components')

# Highlight the highest point
plt.plot(max_x, max_y, 'ro')  # 'ro' makes it a red dot
plt.text(max_x, max_y, f'({max_x}, {max_y:.2f})', ha='left', va='bottom', color='red')

# Add a vertical dotted line
plt.axvline(x=max_x, color='red', linestyle='--', linewidth=1)

# Set x-axis ticks at intervals of 20
plt.xticks(range(0, len(cumulative_variance) + 1, 20))

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# next step is the result of when i trying to find the Different Covariance Types using AIC and BIC as we can see the lowest bar for both blue and oragne is diag

# Step 4 results
plt.figure(figsize=(8, 4))

x_axis_4 = np.arange(len(covariance_types))
plt.bar(x_axis_4 - 0.2, aic_scores_4, 0.4, label="AIC")
plt.bar(x_axis_4 + 0.2, bic_scores_4, 0.4, label="BIC")
plt.xticks(x_axis_4, covariance_types)
plt.xlabel('Covariance Type')
plt.ylabel('AIC/BIC Score')
plt.title('AIC/BIC for Different Covariance Types')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# for this step we found Numbers of Clusters, and here actually used diag covariance try to find Different Numbers of Clusters ..as you can see the minimum point for oragne is 2 (which is for BIC), and minimum pointfor blue is 3 (which is for AIC)
# Step 5 results
plt.figure(figsize=(8, 4))

plt.plot(n_components_range, aic_scores_5, label='AIC', marker='o')
plt.plot(n_components_range, bic_scores_5, label='BIC', marker='o')
plt.xticks(n_components_range)
plt.xlabel('Number of Clusters')
plt.ylabel('AIC/BIC Score')
plt.title(f'AIC/BIC for Different Numbers of Clusters (diag covariance)')
plt.legend()

plt.tight_layout()
plt.show()

## 7. Output the hard clustering assignments for each instance to identify which cluster each image belongs to.

In [ ]:
# Now we try to Output the hard clustering assignments for each instance and we do that by making GaussianMixture variable with this 2 parameters ( n_components = 2 # from BIC, covariance_type = "diag") which is the best parameter we found from above.

# Using parameters from above
n_components = 2 # from BIC
covariance_type = "diag"
n_inits = 10 # here we used same number for inits

gm = GaussianMixture(
    n_components=n_components,
    covariance_type=covariance_type,
    n_init=n_inits,
    random_state=42)
gm.fit(X_train_reduced)

y_train_hard = gm.predict(X_train_reduced)
print(y_train_hard)

counts = np.bincount(y_train_hard)
print(f"Count of 0s: {counts[0]}")
print(f"Count of 1s: {counts[1]}")

# after we print the result, 169 data points that belong to clusters-0 and 151 belongs to clusters-1

#below predecting 120 data point gives me 120 labels

In [ ]:
# I print the shape to chekh if everything is correct, and the shape is 320
y_train_hard.shape

## 8. Output the soft clustering probabilities for each instance to show the likelihood of each image belonging to each cluster.

In [ ]:
# next I find the soft clustering probabilities and this is because we only have to 2 clusters, it should be 2 coloums where 1st coloum is for cluster 0 and 2nd coloum is for cluster 1.

soft_clustering_probabilities = gm.predict_proba(X_train_reduced)

# 3 columns, each column is the likelihood of the image being in a cluster
print(soft_clustering_probabilities)

In [ ]:
#print the shape just to make sure

soft_clustering_probabilities.shape

## 9. Use the model to generate some new faces (using the sample() method) and visualize them (use the inverse_transform() method to transform the data back to its original space based on the PCA method used).

In [ ]:
# I used the model to Generated new faces using gm.sample of 5 faces

X_new_reduced, y_new_reduced = gm.sample(5)
print("Generated new points:\n", X_new_reduced)
print("Generated new labels:\n", y_new_reduced)

In [ ]:
X_new_original = pca.inverse_transform(X_new_reduced)

fig, axarr = plt.subplots(nrows=1, ncols=5, figsize=(18,9))
axarr = axarr.flatten()
for i in range(5):
    axarr[i].imshow(X_new_original[i].reshape(64, 64), cmap='gray')
    axarr[i].set_xticks([])
    axarr[i].set_yticks([])
    axarr[i].set_title(f"cluster id: {y_new_reduced[i]}")

## 10. Modify some images (e.g., rotate, flip, darken).

In [ ]:
# Next I Modify some images

X_modified = []
random.seed(42) # for random angles

# Modify first 5 images in training set
for i in range(5):
    original_image = X_train[i].reshape(64, 64)

    ## Rotate image (random angles between -180 and 180)
    modified_image = rotate(original_image, angle=random.randint(-180, 180), reshape=False)

    ##Flip horizontally
    modified_image = np.flip(modified_image, axis=1)

    ## Darken
    modified_image = modified_image - 0.45

    ## Ensure values are still between 0 and 1
    modified_image = np.clip(modified_image, 0, 1)

    X_modified.append(modified_image)

# Plot unmodified vs modified
fig, axarr = plt.subplots(nrows=2, ncols=5, figsize=(18,9))
axarr = axarr.flatten()
for i in range(5):
    axarr[i].imshow(X_train[i].reshape(64, 64), cmap='gray')
    axarr[i].set_xticks([])
    axarr[i].set_yticks([])
    axarr[i].set_title(f"person id: {y_train[i]}")
for i in range(5, 10):
    axarr[i].imshow(X_modified[i-5], cmap='gray')
    axarr[i].set_xticks([])
    axarr[i].set_yticks([])
    axarr[i].set_title(f"person id: {y_train[i-5]}")
plt.suptitle("Original vs Modified")
plt.tight_layout()
plt.show()

# here i plot 1st 5 images from train images
# we can see that person id:31 image, which after modification we can see on 2nd row

## 11. Determine if the model can detect the anomalies produced in step 10 by comparing the output of the score_samples() method for normal images and for anomalies.

In [ ]:
# Last we check if we can detect the anomalies
# Initialize variables
X_normals = X_train[:5]
X_anomalies = np.array(X_modified).reshape((5, 4096))
print(X_normals.shape)
print(X_anomalies.shape)

In [ ]:
# Apply PCA ( we used same PCA as above ) so it fits the model
X_normals_reduced = pca.transform(X_normals)
X_anomalies_reduced = pca.transform(X_anomalies)
print(X_normals_reduced.shape)
print(X_anomalies_reduced.shape)

In [ ]:
# Get scores
normal_scores = gm.score_samples(X_normals_reduced)
anomaly_scores = gm.score_samples(X_anomalies_reduced)

In [ ]:
normal_scores

In [ ]:
anomaly_scores

- After we print the normal_scores and anomaly_scores we can see both scores are very diffrent and anomaly_scores is much lower then the normal_scores


- I belive it seems to be able to detect the anomalies. the scores of the modified images are much lower compared to the images that are unmodified.

> Add blockquote

